In [1]:
import pandas as pd
import numpy as np


In [2]:
translations = pd.read_csv("training.txt", sep="\t", header=None, names=['labels','phrases'])
translations

,labels,phrases
0,1,"La noticia de WhatsApp , aplicación de mensaje..."
1,0,"En cualquier caso , aunque la respeto mucho Fr..."
2,0,Entre los 320 individuos de actuación son el m...
3,1,Tal vez ella vaya a tomar aunque sea alguno de...
4,1,Ella ha hecho mucho por las mujeres en la músi...
5,0,"Astro Teller tiene su trabajo ideal , actualme..."
6,0,Nueve de los socios de Accel han realizado inv...
7,1,"Otras veces , parecen guíarse de Hollywood .\..."
8,0,La lectura de las despedidas mis compañeros bl...
9,0,Incluso tratando de mirar a la otra persona a ...


In [8]:
#Notes about the data:
# it's about 50/50 labels

translations['phrases'].values

array([ 'La noticia de WhatsApp , aplicaci\xc3\xb3n de mensajer\xc3\xada m\xc3\xa1s grande del mundo , que fue adquirido el 19 de febrero por la red social m\xc3\xa1s grande del mundo tambi\xc3\xa9n se ha dado a luz a una nueva estrella para la App Store .',
       'En cualquier caso , aunque la respeto mucho Fred , perm\xc3\xadtanme exponer mi posici\xc3\xb3n para el registro - los consumidores y las peque\xc3\xb1as empresas de Internet oportunidades est\xc3\xa1n lejos de ser " sobre " , y cualquier comentario es una tonter\xc3\xada , de mente estrecha , y simplemente equivocado ( Altho , puede de hecho ser " sobre " para la mayor\xc3\xada de los grandes fondos que tratan de hacer de semillas en fase de inversi\xc3\xb3n en cualquier escala similar a TA o 500 ) .',
       'Entre los 320 individuos de actuaci\xc3\xb3n son el ministro del Interior , Thomas de Maizi\xc3\xa8re .',
       ...,
       'Y \xc3\xa9l dice que podr\xc3\xada hacerlo 200X mayor de la bater\xc3\xada a base de litio

In [10]:
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import sklearn
from stop_words import get_stop_words
from time import time



In [11]:
es_stop = get_stop_words('es')


In [12]:
M_doc_freq = 0.99
min_doc_freq = 3

CV = CountVectorizer(max_df=M_doc_freq, min_df=min_doc_freq, stop_words=es_stop, binary=True, lowercase=True)

In [14]:
# Fit the vectorizer
t0 = time()
CV = CV.fit(translations['phrases'].values)

phrase_matrix = CV.transform(translations['phrases'].values)
print("done in %0.3fs." % (time() - t0))
tf_feature_names = CV.get_feature_names()

done in 1.277s.


In [15]:
len(CV.vocabulary_)

11305

In [16]:
CV.vocabulary_

{u'woods': 11197,
 u'woody': 11198,
 u'agreg\xf3': 470,
 u'form\xf3': 4720,
 u'igual': 5493,
 u'ciudades': 2035,
 u't\xeda': 10663,
 u'observadores': 7437,
 u'hermana': 5287,
 u'hermano': 5289,
 u't\xedo': 10666,
 u'dictada': 3353,
 u'lumia': 6546,
 u'minoristas': 6963,
 u'cibern\xe9tico': 1981,
 u'caramelo': 1765,
 u'camisa': 1692,
 u'dubl\xedn': 3622,
 u'importaci\xf3n': 5540,
 u'locos': 6484,
 u'widget': 11172,
 u'culo': 2850,
 u'socavar': 9844,
 u'probabilidad': 8442,
 u'lgbt': 6313,
 u'china': 1959,
 u'pu\xf1ado': 8661,
 u'chino': 1961,
 u'visualizaci\xf3n': 11044,
 u'enganchado': 3950,
 u'caro': 1792,
 u'declarar': 3022,
 u'escribieron': 4113,
 u'stern': 9985,
 u'experimentaci\xf3n': 4401,
 u'alimentado': 567,
 u'music': 7165,
 u'posteriormente': 8267,
 u'implicaci\xf3n': 5534,
 u'yahoo': 11212,
 u'destruir': 3289,
 u'luego': 6538,
 u'pac\xedfico': 7666,
 u'40': 97,
 u'temas': 10272,
 u'adquirir': 395,
 u'rayos': 8778,
 u'fabricantes': 4482,
 u'etiqueta': 4303,
 u'absoluto': 192,

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import csv

In [18]:
data = phrase_matrix
target = translations['labels'].values
classf =  SVC()

In [20]:
t0 = time()
classf.fit(data,target)
print "Score of SVM: " + str(classf.score(data,target))
print("done in %0.3fs." % (time() - t0))

Score of SVM: 0.520068671561
done in 116.832s.


In [19]:
t0 = time()
svd = TruncatedSVD(n_components=100, random_state=42)
svd.fit(data) 
data = svd.transform(data)
print("SVD fit done in %0.3fs." % (time() - t0))

SVD fit done in 1.538s.


In [ ]:
t0 = time()
data = data
pca = PCA(n_components =500)
pca.fit(data)
data = pca.transform(data)
print("PCA fit done in %0.3fs." % (time() - t0))

In [40]:
t0 = time()
classf = SVC()
scores = cross_validation.cross_val_score(classf,data,target, n_jobs=-1, cv = 10)
print "Scores: " + str(scores)
print "Crossval Final score: "+ str(sum(scores)/10)
print("Cross val done in %0.3fs." % (time() - t0))

[ 0.50473373  0.51775148  0.51715976  0.51213736  0.51865009  0.53108348
  0.5062167   0.50562463  0.51154529  0.50236967]
Cross val done in 330.714s.


0.51272721875061178

In [34]:
# How to process a test set and predict the labels 

test = pd.read_csv("example_test.txt", sep="\t", header=None, names=['labels','phrases'], lineterminator='\n', quoting=csv.QUOTE_NONE)
test

,labels,phrases
0,_,""" Otras veces , parecen guíarse de Hollywood ."
1,_,Apuestas JayBird En seguimiento inteligente pa...
2,_,"En esta ocasión , ¡ nos hemos dado cuenta !"
3,_,Esta es noticia loca !
4,_,No es una mala combinación si se va a tomar ta...
5,_,"Inicio Adventr ( pronunciado "" aventura "" ) es..."
6,_,La lectura de las despedidas mis compañeros bl...
7,_,Incluso tratando de mirar a la otra persona a ...
8,_,Los dejo con el mejor discurso presidencial ja...
9,_,Las 23 Cosas Todas las mujeres piensan de penes


In [35]:
testlabs = classf.predict(svd.transform(CV.transform(test['phrases'])))

In [36]:
test.labels = test['labels'] = testlabs
test

,labels,phrases
0,1,""" Otras veces , parecen guíarse de Hollywood ."
1,1,Apuestas JayBird En seguimiento inteligente pa...
2,1,"En esta ocasión , ¡ nos hemos dado cuenta !"
3,1,Esta es noticia loca !
4,1,No es una mala combinación si se va a tomar ta...
5,0,"Inicio Adventr ( pronunciado "" aventura "" ) es..."
6,1,La lectura de las despedidas mis compañeros bl...
7,1,Incluso tratando de mirar a la otra persona a ...
8,1,Los dejo con el mejor discurso presidencial ja...
9,1,Las 23 Cosas Todas las mujeres piensan de penes


In [39]:
test.to_csv("example_test_output.txt", sep="\t", header=False, index=False, quoting=csv.QUOTE_NONE)

In [27]:
test.iloc[0]['phrases']

' Otras veces , parecen gu\xc3\xadarse de Hollywood .\n_\tApuestas JayBird En seguimiento inteligente para el reino , su primera incursi\xc3\xb3n en Quantified fitness\n_\tEn esta ocasi\xc3\xb3n , \xc2\xa1 nos hemos dado cuenta !\n_\tEsta es noticia loca !\n_\tNo es una mala combinaci\xc3\xb3n si se va a tomar tanto en Sonos y otros reproductores de automatizaci\xc3\xb3n del hogar , como el nido y muchos otros .\n_\tInicio Adventr ( pronunciado  aventura " ) est\xc3\xa1 apostando a que puede hacer que el v\xc3\xaddeo en l\xc3\xadnea sea incluso m\xc3\xa1s adictivo con la interactividad .'

In [4]:
import nltk
from nltk.tag import StanfordPOSTagger
from nltk.tokenize import TweetTokenizer
from nltk.util import ngrams
from collections import Counter

import itertools
import string

In [5]:
#tknzr = TweetTokenizer()
#words = tknzr.tokenize(s0.lower())


s0 = "AQ AO RG DD DP DT DE DI DA NC NP VM VS VA PP PD PX PI PT PR PE CC CS I SP F Z"
words = s0.lower().split()
print words

['aq', 'ao', 'rg', 'dd', 'dp', 'dt', 'de', 'di', 'da', 'nc', 'np', 'vm', 'vs', 'va', 'pp', 'pd', 'px', 'pi', 'pt', 'pr', 'pe', 'cc', 'cs', 'i', 'sp', 'f', 'z']


In [6]:
fdist = itertools.combinations(words, 2)
#print [a for a in fdist]
dic = {}
for i,k in enumerate(fdist):
    dic[k]=i
    
print dic

dic[('nc','cc')]


{('nc', 'cc'): 209, ('aq', 'i'): 22, ('pd', 'sp'): 293, ('dd', 'pr'): 90, ('vm', 'pd'): 234, ('dd', 'dp'): 75, ('pt', 'f'): 321, ('dt', 'pp'): 128, ('pd', 'px'): 285, ('di', 'pp'): 167, ('px', 'sp'): 303, ('ao', 'z'): 50, ('cc', 'z'): 340, ('da', 'z'): 197, ('aq', 'dd'): 2, ('de', 'pp'): 148, ('nc', 'z'): 214, ('rg', 'vm'): 59, ('aq', 'px'): 15, ('pp', 'i'): 281, ('dt', 'i'): 137, ('aq', 'z'): 25, ('vm', 'f'): 244, ('pr', 'pe'): 323, ('da', 'cs'): 193, ('ao', 'pt'): 42, ('pd', 'i'): 292, ('vs', 'pr'): 252, ('aq', 'rg'): 1, ('ao', 'dp'): 28, ('di', 'np'): 163, ('cc', 'cs'): 336, ('vm', 'vs'): 231, ('pp', 'f'): 283, ('px', 'i'): 302, ('pd', 'f'): 294, ('de', 'np'): 144, ('dp', 'z'): 119, ('pp', 'pt'): 276, ('dd', 'de'): 77, ('aq', 'dp'): 3, ('px', 'pt'): 297, ('de', 'pd'): 149, ('vm', 'pi'): 236, ('dt', 'de'): 120, ('np', 'pp'): 218, ('rg', 'pd'): 63, ('pi', 'cc'): 309, ('aq', 'cs'): 21, ('ao', 'de'): 30, ('np', 'cc'): 225, ('pr', 'sp'): 327, ('dt', 'nc'): 123, ('pe', 'i'): 332, ('nc', '

209

In [12]:
tagers=[]
st = StanfordPOSTagger('/home/jmirandadealme/stanford_nlp/stanford-postagger-full-2015-12-09/models/spanish.tagger',
                       '/home/jmirandadealme/stanford_nlp/stanford-postagger-full-2015-12-09/stanford-postagger.jar')
tagers = st.tag('a noticia de WhatsApp , aplicación de mensajería más grande del mundo , que fue adquirido el 19 de febrero por la red social más grande del mundo también se ha dado a luz a una nueva estrella para la App Store .'.split()) 


Exception in thread "main" java.lang.UnsupportedClassVersionError: edu/stanford/nlp/tagger/maxent/MaxentTagger : Unsupported major.minor version 52.0
	at java.lang.ClassLoader.defineClass1(Native Method)
	at java.lang.ClassLoader.defineClass(ClassLoader.java:800)
	at java.security.SecureClassLoader.defineClass(SecureClassLoader.java:142)
	at java.net.URLClassLoader.defineClass(URLClassLoader.java:449)
	at java.net.URLClassLoader.access$100(URLClassLoader.java:71)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:361)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:355)
	at java.security.AccessController.doPrivileged(Native Method)
	at java.net.URLClassLoader.findClass(URLClassLoader.java:354)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:425)
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:308)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:358)
	at sun.launcher.LauncherHelper.checkAndLoadMain(LauncherHelper.java:482)



OSError: Java command failed : [u'/usr/bin/java', '-mx1000m', '-cp', '/home/jmirandadealme/stanford_nlp/stanford-postagger-full-2015-12-09/stanford-postagger.jar', 'edu.stanford.nlp.tagger.maxent.MaxentTagger', '-model', '/home/jmirandadealme/stanford_nlp/stanford-postagger-full-2015-12-09/models/spanish.tagger', '-textFile', '/tmp/tmpq0dN00', '-tokenize', 'false', '-outputFormatOptions', 'keepEmptySentences', '-encoding', 'utf8']

In [8]:
line = []
for k in tagers:
    for n in words:
        if( n in k[1]):
            print k[0] + " -> " + n + " in " + k[1]
            line.append(n)

In [9]:
print line
bigrams=ngrams(line,2)
print Counter(bigrams)

[]
Counter()


In [10]:
def postag(txt,Dictio=dic,tagger=st):
    vect = [0] * len(dic)
    txt = ''.join([c for c in txt if c not in string.punctuation])
    txt_lst = txt.split()
    for wrd in txt_lst:
        vect[Diction[st.tag(txt_lst)]] +=1 

NameError: name 'st' is not defined

In [28]:
postag('a noticia de WhatsApp , aplicación de mensajería más grande del mundo')

[(u'a', u'sp000'), (u'n', u'nc0n000'), (u'o', u'cc'), (u't', u'np00000'), (u'i', u'nc0s000'), (u'c', u'np00000'), (u'i', u'nc0s000'), (u'a', u'sp000'), (u'd', u'np00000'), (u'e', u'cc'), (u'W', u'np00000'), (u'h', u'nc0s000'), (u'a', u'sp000'), (u't', u'np00000'), (u's', u'np00000'), (u'A', u'np00000'), (u'p', u'np00000'), (u'p', u'np00000'), (u',', u'fc'), (u'a', u'sp000'), (u'p', u'nc0n000'), (u'l', u'nc0n000'), (u'i', u'nc0s000'), (u'c', u'np00000'), (u'a', u'sp000'), (u'c', u'np00000'), (u'i', u'nc0s000'), (u'\ufffd', u'np00000'), (u'\ufffd', u'np00000'), (u'n', u'nc0n000'), (u'd', u'np00000'), (u'e', u'cc'), (u'm', u'np00000'), (u'e', u'cc'), (u'n', u'nc0n000'), (u's', u'nc0n000'), (u'a', u'sp000'), (u'j', u'nc0s000'), (u'e', u'cc'), (u'r', u'np00000'), (u'\ufffd', u'f0'), (u'\ufffd', u'vsis000'), (u'a', u'sp000'), (u'm', u'np00000'), (u'\ufffd', u'np00000'), (u'\ufffd', u'np00000'), (u's', u'nc0n000'), (u'g', u'nc0n000'), (u'r', u'nc0n000'), (u'a', u'sp000'), (u'n', u'nc0n000'), 